<a href="https://colab.research.google.com/github/Anjasfedo/Code-as-a-Cryptography/blob/main/ecc_lsb_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECC

In [1]:
!pip install eciespy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.6 MB/s eta 0:00:00


# LSB

In [2]:
!git clone https://github.com/RobinDavid/LSB-Steganography && cd LSB-Steganography && ls && pip install -r requirements.txt

Cloning into 'LSB-Steganography'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 75 (delta 1), reused 2 (delta 0), pack-reused 68 (from 1)
Receiving objects: 100% (75/75), 19.72 KiB | 9.86 MiB/s, done.
Resolving deltas: 100% (26/26), done.
LICENCE  LSBSteg.py  README.md	requirements.txt
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=2625beabd5e7df3ae8e69362bd5e268e5bd7eff8beaaf87de9970edf69f25411
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:
#!/usr/bin/env python
# coding:UTF-8
"""LSBSteg.py

Usage:
  LSBSteg.py encode -i <input> -o <output> -f <file>
  LSBSteg.py decode -i <input> -o <output>

Options:
  -h, --help                Show this help
  --version                 Show the version
  -f,--file=<file>          File to hide
  -i,--in=<input>           Input image (carrier)
  -o,--out=<output>         Output image (or extracted file)
"""

import cv2
import docopt
import numpy as np


class SteganographyException(Exception):
    pass


class LSBSteg():
    def __init__(self, im):
        self.image = im
        self.height, self.width, self.nbchannels = im.shape
        self.size = self.width * self.height

        self.maskONEValues = [1,2,4,8,16,32,64,128]
        #Mask used to put one ex:1->00000001, 2->00000010 .. associated with OR bitwise
        self.maskONE = self.maskONEValues.pop(0) #Will be used to do bitwise operations

        self.maskZEROValues = [254,253,251,247,239,223,191,127]
        #Mak used to put zero ex:254->11111110, 253->11111101 .. associated with AND bitwise
        self.maskZERO = self.maskZEROValues.pop(0)

        self.curwidth = 0  # Current width position
        self.curheight = 0 # Current height position
        self.curchan = 0   # Current channel position

    def put_binary_value(self, bits): #Put the bits in the image
        for c in bits:
            val = list(self.image[self.curheight,self.curwidth]) #Get the pixel value as a list
            if int(c) == 1:
                val[self.curchan] = int(val[self.curchan]) | self.maskONE #OR with maskONE
            else:
                val[self.curchan] = int(val[self.curchan]) & self.maskZERO #AND with maskZERO

            self.image[self.curheight,self.curwidth] = tuple(val)
            self.next_slot() #Move "cursor" to the next space

    def next_slot(self):#Move to the next slot were information can be taken or put
        if self.curchan == self.nbchannels-1: #Next Space is the following channel
            self.curchan = 0
            if self.curwidth == self.width-1: #Or the first channel of the next pixel of the same line
                self.curwidth = 0
                if self.curheight == self.height-1:#Or the first channel of the first pixel of the next line
                    self.curheight = 0
                    if self.maskONE == 128: #Mask 1000000, so the last mask
                        raise SteganographyException("No available slot remaining (image filled)")
                    else: #Or instead of using the first bit start using the second and so on..
                        self.maskONE = self.maskONEValues.pop(0)
                        self.maskZERO = self.maskZEROValues.pop(0)
                else:
                    self.curheight +=1
            else:
                self.curwidth +=1
        else:
            self.curchan +=1

    def read_bit(self): #Read a single bit int the image
        val = self.image[self.curheight,self.curwidth][self.curchan]
        val = int(val) & self.maskONE
        self.next_slot()
        if val > 0:
            return "1"
        else:
            return "0"

    def read_byte(self):
        return self.read_bits(8)

    def read_bits(self, nb): #Read the given number of bits
        bits = ""
        for i in range(nb):
            bits += self.read_bit()
        return bits

    def byteValue(self, val):
        return self.binary_value(val, 8)

    def binary_value(self, val, bitsize): #Return the binary value of an int as a byte
        binval = bin(val)[2:]
        if len(binval) > bitsize:
            raise SteganographyException("binary value larger than the expected size")
        while len(binval) < bitsize:
            binval = "0"+binval
        return binval

    def encode_text(self, txt):
        l = len(txt)
        binl = self.binary_value(l, 16) #Length coded on 2 bytes so the text size can be up to 65536 bytes long
        self.put_binary_value(binl) #Put text length coded on 4 bytes
        for char in txt: #And put all the chars
            c = ord(char)
            self.put_binary_value(self.byteValue(c))
        return self.image

    def decode_text(self):
        ls = self.read_bits(16) #Read the text size in bytes
        l = int(ls,2)
        i = 0
        unhideTxt = ""
        while i < l: #Read all bytes of the text
            tmp = self.read_byte() #So one byte
            i += 1
            unhideTxt += chr(int(tmp,2)) #Every chars concatenated to str
        return unhideTxt

    def encode_image(self, imtohide):
        w = imtohide.width
        h = imtohide.height
        if self.width*self.height*self.nbchannels < w*h*imtohide.channels:
            raise SteganographyException("Carrier image not big enough to hold all the datas to steganography")
        binw = self.binary_value(w, 16) #Width coded on to byte so width up to 65536
        binh = self.binary_value(h, 16)
        self.put_binary_value(binw) #Put width
        self.put_binary_value(binh) #Put height
        for h in range(imtohide.height): #Iterate the hole image to put every pixel values
            for w in range(imtohide.width):
                for chan in range(imtohide.channels):
                    val = imtohide[h,w][chan]
                    self.put_binary_value(self.byteValue(int(val)))
        return self.image


    def decode_image(self):
        width = int(self.read_bits(16),2) #Read 16bits and convert it in int
        height = int(self.read_bits(16),2)
        unhideimg = np.zeros((width,height, 3), np.uint8) #Create an image in which we will put all the pixels read
        for h in range(height):
            for w in range(width):
                for chan in range(unhideimg.channels):
                    val = list(unhideimg[h,w])
                    val[chan] = int(self.read_byte(),2) #Read the value
                    unhideimg[h,w] = tuple(val)
        return unhideimg

    def encode_binary(self, data):
        l = len(data)
        if self.width*self.height*self.nbchannels < l+64:
            raise SteganographyException("Carrier image not big enough to hold all the datas to steganography")
        self.put_binary_value(self.binary_value(l, 64))
        for byte in data:
            byte = byte if isinstance(byte, int) else ord(byte) # Compat py2/py3
            self.put_binary_value(self.byteValue(byte))
        return self.image

    def decode_binary(self):
        l = int(self.read_bits(64), 2)
        output = b""
        for i in range(l):
            output += bytearray([int(self.read_byte(),2)])
        return output


def main():
    args = docopt.docopt(__doc__, version="0.2")
    in_f = args["--in"]
    out_f = args["--out"]
    in_img = cv2.imread(in_f)
    steg = LSBSteg(in_img)
    lossy_formats = ["jpeg", "jpg"]

    if args['encode']:
        #Handling lossy format
        out_f, out_ext = out_f.split(".")
        if out_ext in lossy_formats:
            out_f = out_f + ".png"
            print("Output file changed to ", out_f)

        data = open(args["--file"], "rb").read()
        res = steg.encode_binary(data)
        cv2.imwrite(out_f, res)

    elif args["decode"]:
        raw = steg.decode_binary()
        with open(out_f, "wb") as f:
            f.write(raw)

# if __name__=="__main__":
#     main()

In [4]:
CONTENT = "3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup"

In [5]:
LENA_IMG = 'ori-image/lena.png'

In [6]:
def embed_lsb(target, content, ori_img = LENA_IMG):
  steg = LSBSteg(cv2.imread(ori_img))

  img_encoded = steg.encode_text(content)

  lsb_img = f"{stego_dir_img}/{target}"

  cv2.imwrite(lsb_img, img_encoded)

In [7]:
def extract_lsb(target):
  im = cv2.imread(f"{stego_dir_img}/{target}")

  steg = LSBSteg(im)

  content = steg.decode_text()

  return content

In [8]:
# embed_lsb("lsb_only.png", CONTENT)

In [9]:
# extract_lsb("lsb_only.png")

In [10]:
# calculate_psnr_2("lsb_only.png")

## LSB with stego package

In [11]:
!pip install stegano

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 2.0 MB/s eta 0:00:00


In [36]:
from stegano import lsb
# CONTENT2="dˆ|&0%|0*6c#|8z ̃c7c#zm8d*0*08*8p8$*cd,d+zez,|6z)8!cq zb**zzcq8jcazqz!**dkds*dcacl*n|6z)0y0/cq7)8=*ld1c#7q *9d08}0|0:7.*08w8$dacl0+zez?z=8=cbze|{cjc[db8m0]07c, 0e8mzq700x*2cdc+z)c2**8 ̃dz7_c7|88p|{0|0_*78$*mza8+8= d(0i*m||7gza0jc/8=c\d18f8jzqzi|9|68$7ndi7g8(|g*lcazr 0ycjdm*l0,7l07zrdi0&zec+0yd\8l8m0e7r7m7!76*mclz58=z, d9z,83*ac[|%dfze0yz,dm0,09dsd+07*h0mc|zhzs"

# secret=lsb.hide("/content/ori-image/lena.png",CONTENT2)

# secret.save("/content/stego-image/Encoded_Image.png")

In [13]:
# print(lsb.reveal("/content/stego-image/Encoded_Image.png"))

In [14]:
# import base64
# from stegano import lsb

# # Your original content with special characters
# CONTENT2 = "dˆ|&0%|0*6c#|8z ̃c7c#zm8d*0*08*8p8$*cd,d+zez,|6z)8!cq zb**zzcq8jcazqz!**dkds*dcacl*n|6z)0y0/cq7)8=*ld1c#7q *9d08}0|0:7.*08w8$dacl0+zez?z=8=cbze|{cjc[db8m0]07c, 0e8mzq700x*2cdc+z)c2**8 ̃dz7_c7|88p|{0|0_*78$*mza8+8= d(0i*m||7gza0jc/8=c\d18f8jzqzi|9|68$7ndi7g8(|g*lcazr 0ycjdm*l0,7l07zrdi0&zec+0yd\8l8m0e7r7m7!76*mclz58=z, d9z,83*ac[|%dfze0yz,dm0,09dsd+07*h0mc|zhzs"

# # Encode CONTENT2 in base64 to make it safe for embedding
# CONTENT2_base64 = base64.b64encode(CONTENT2.encode('utf-8')).decode('utf-8')
# print(CONTENT2_base64)
# print(base64.b64decode(CONTENT2_base64).decode('utf-8'))

# # Hide the base64-encoded content inside the image
# secret = lsb.hide("/content/ori-image/lena.png", CONTENT2_base64)

# # Save the stego image
# secret.save("/content/stego-image/Encoded_Image.png")

# # Reveal the hidden content from the stego image
# hidden_content = lsb.reveal("/content/stego-image/Encoded_Image.png")

# # Decode the base64 content back to the original string
# decoded_content = base64.b64decode(hidden_content).decode('utf-8')

# print("Decoded Content:", decoded_content)


In [15]:
# calculate_psnr_2("Encoded_Image.png")

In [16]:

# from math import log10, sqrt
# import cv2
# import numpy as np

# def PSNR(original, compressed, max_value=255):
#     mse = np.mean((np.array(original, dtype=np.float32) - np.array(compressed, dtype=np.float32)) ** 2)
#     if mse == 0:
#         return 100
#     return 20 * np.log10(max_value / (np.sqrt(mse)))


# original = cv2.imread("/content/ori-image/lena.png")
# compressed = cv2.imread("/content/stego-image/Encoded_Image.png", 1)
# value = PSNR(original, compressed)
# print(f"PSNR value is {value} dB")

In [17]:
# import cv2
# img1 = cv2.imread('/content/ori-image/lena.png')
# img2 = cv2.imread('/content/stego-image/Encoded_Image.png')
# psnr = cv2.PSNR(img1, img2)
# psnr

# Huffman

In [18]:
import sys

class Letter:
    def __init__(self, letter, freq):
        self.letter = letter
        self.freq = freq
        self.bitstring = ""

    def __repr__(self):
        return f'{self.letter}:{self.freq}'


class TreeNode:
    def __init__(self, freq, left, right):
        self.freq = freq
        self.left = left
        self.right = right


def parse_file(file_path):
    """
    Read the file and build a dict of all letters and their
    frequences, then convert the dict into a list of Letters.
    """
    chars = {}
    with open(file_path) as f:
        while True:
            c = f.read(1)
            if not c:
                break
            chars[c] = chars[c] + 1 if c in chars.keys() else 1
    return sorted([Letter(c, f) for c, f in chars.items()], key=lambda l: l.freq)

def build_tree(letters):
    """
    Run through the list of Letters and build the min heap
    for the Huffman Tree.
    """
    while len(letters) > 1:
        left = letters.pop(0)
        right = letters.pop(0)
        total_freq = left.freq + right.freq
        node = TreeNode(total_freq, left, right)
        letters.append(node)
        letters.sort(key=lambda l: l.freq)
    return letters[0]

def traverse_tree(root, bitstring):
    """
    Recursively traverse the Huffman Tree to set each
    Letter's bitstring, and return the list of Letters
    """
    if type(root) is Letter:
        root.bitstring = bitstring
        return [root]
    letters = []
    letters += traverse_tree(root.left, bitstring + "0")
    letters += traverse_tree(root.right, bitstring + "1")
    return letters

def huffman(file_path):
    """
    Parse the file, build the tree, then run through the file
    again, using the list of Letters to find and print out the
    bitstring for each letter.
    """
    letters_list = parse_file(file_path)
    root = build_tree(letters_list)
    letters = traverse_tree(root, "")
    print(f'Huffman Coding of {file_path}: ')
    with open(file_path) as f:
        while True:
            c = f.read(1)
            if not c:
                break
            le = list(filter(lambda l: l.letter == c, letters))[0]
            print(le.bitstring, end=" ")
    print()

# Create and write to a file
file_path = "/content/sample_file.txt"

with open(file_path, 'w') as f:
    f.write("This is an example file.\n")
    f.write("Huffman coding example input.\n")
    f.write("hello world!\n")

print(f"File created at: {file_path}")


huffman("/content/sample_file.txt")

File created at: /content/sample_file.txt
Huffman Coding of /content/sample_file.txt: 
101000 01110 1100 01111 010 1100 01111 010 0011 0110 010 1101 10000 0011 11110 11111 1110 1101 010 0000 1100 1110 1101 10001 0001 101001 10010 0000 0000 11110 0011 0110 010 101010 0010 10011 1100 0110 101011 010 1101 10000 0011 11110 11111 1110 1101 010 1100 0110 11111 10010 101100 10001 0001 01110 1101 1110 1110 0010 010 101101 0010 101110 1110 10011 101111 0001 


In [19]:
!pip install steg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Pillow: filename=Pillow-8.1.2-cp310-cp310-linux_x86_64.whl size=1163963 sha256=659d04daa171ca80d279a090313d6e82b2700cd4675d6f0c81ef454924181dac
  Stored in directory: /root/.cache/pip/wheels/b1/fd/2b/ad2015f7e099ff6596b21bb829692ccaef4d111f82549448bd
Successfully built Pillow
  Attempting uninstall: Pillow
    Found existing installation: pillow 10.4.0
    Uninstalling pillow-10.4.0:
      Successfully uninstalled pillow-10.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.17 requires pillow>=9.0.0, but you have pillow 8.1.2 which is incompatible.
imageio 2.35.1 requires pillow>=8.3.2, but you have pillow 8.1.2 which is incompatible.
scikit-image 0.24.0 requires pillow>=9.1, but you have pillow 8.1.2 which is 

# PSNR

In [20]:
def calculate_psnr_2(target, ori_image = LENA_IMG):
    # Read images
    original = cv2.imread(ori_image)
    stego = cv2.imread(f'{stego_dir_img}/{target}')

    if original is None:
        raise ValueError(f"Failed to load original image from {ori_image}. Ensure the file exists and is a valid image format.")
    if stego is None:
        raise ValueError(f"Failed to load stego image from {stego_dir_img}/{target}. Ensure the file exists and is a valid image format.")

    # Calculate MSE (Mean Squared Error)
    mse = np.mean((original - stego) ** 2)
    if mse == 0:  # If images are identical
        return float('inf')

    # Calculate PSNR using the 10 log10 version
    max_pixel_value = 255.0
    psnr = 10 * np.log10((max_pixel_value ** 2) / mse)
    print(f'PSNR: {psnr}')
    return psnr

# Test Image

In [21]:
import os
# Directory where the image will be saved
ori_dir_img = "ori-image"

# Create the ori_dir_img if it doesn't exist
if not os.path.exists(ori_dir_img):
    os.makedirs(ori_dir_img)

stego_dir_img = "stego-image"

# Create the stego_dir_img if it doesn't exist
if not os.path.exists(stego_dir_img):
    os.makedirs(stego_dir_img)

## Lena Image

In [22]:
import requests
from PIL import Image
import io

# URL to the raw image file
url = "https://raw.githubusercontent.com/mikolalysenko/lena/master/lena.png"

# Download the image
response = requests.get(url)
if response.status_code == 200:
    # Load the image using PIL
    lena_image = Image.open(io.BytesIO(response.content))
    lena_image.show()  # Display the image (optional)
    lena_image.save("ori-image/lena.png")  # Save the image locally
else:
    print("Failed to download the image.")

In [23]:
LENA_IMG = 'ori-image/lena.png'

if not os.path.exists(LENA_IMG):
    raise FileNotFoundError(f"Image not found at {LENA_IMG}")

# LSB Only

In [24]:
CONTENT = "3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup"

embed_lsb("lsb_only.png", CONTENT)

In [25]:
extract_lsb("lsb_only.png")

'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup'

In [26]:
calculate_psnr_2("lsb_only.png")

PSNR: 78.57483188820159


78.57483188820159

# LSB with ECC

In [27]:
from ecies.utils import generate_eth_key
from ecies import encrypt, decrypt

private_key = generate_eth_key()
public_key = private_key.public_key

public_key_hex = public_key.to_hex()
private_key_hex = private_key.to_hex()

print("Public Key:", public_key_hex)
print("Private Key:", private_key_hex)

ciphertext = encrypt(public_key_hex, CONTENT.encode())

print("Ciphertext:", ciphertext.hex())

decrypted_message = decrypt(private_key_hex, ciphertext)
print("Decrypted Message:", decrypted_message.decode())

Public Key: 0x06d3c69898a499bd3d7d83a1b0e05d05c538584c80c60e38dcb3b34895977ef33cea1406ecba85f92420559f1e5b36cd912b19c941561fb2b0f4aa244db43138
Private Key: 0x2b7386da85f4eab078975dd2e481ff4ba913d08e5b541a04489345301750d18d
Ciphertext: 049e00e9775dda55f9368c3e84de30071257d536427dc6bfd2de888473a7914c3bb33504c598a5b33a172f400a4590f41fb8b5da18f5804d205e8aba2f692facacb4787713de93dfaa218e952ef5c368dcee97678cc587996dcaf57644f7b89d9d6d633e83d9a15e5d0888774d3b4fe3e620b66ece887ae5df70beeba5894fc4b0674f51e8574671e747b9b8d7ef5a73e386e2e7cb1f7ba15fd8ea03aec50b09b2611d06d445333bca525e10459c8e5b1ceaa7e3deb78f9cd49d40e37d4d436c8978dbc2419fbd54d79f7f4ad3e41f6275c1e92e08cb9a7d9a71940f4f79e1d9a608edc06b1134087ba5a560a06384ec8f589f334b379d612b455e9da5ce8da353c0ac0130fa7b57779b41dbc978c13d04c0
Decrypted Message: 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup


In [28]:
print(ciphertext)
print(ciphertext.hex())

b'\x04\x9e\x00\xe9w]\xdaU\xf96\x8c>\x84\xde0\x07\x12W\xd56B}\xc6\xbf\xd2\xde\x88\x84s\xa7\x91L;\xb35\x04\xc5\x98\xa5\xb3:\x17/@\nE\x90\xf4\x1f\xb8\xb5\xda\x18\xf5\x80M ^\x8a\xba/i/\xac\xac\xb4xw\x13\xde\x93\xdf\xaa!\x8e\x95.\xf5\xc3h\xdc\xee\x97g\x8c\xc5\x87\x99m\xca\xf5vD\xf7\xb8\x9d\x9dmc>\x83\xd9\xa1^]\x08\x88wM;O\xe3\xe6 \xb6n\xce\x88z\xe5\xdfp\xbe\xeb\xa5\x89O\xc4\xb0gOQ\xe8WFq\xe7G\xb9\xb8\xd7\xefZs\xe3\x86\xe2\xe7\xcb\x1f{\xa1_\xd8\xea\x03\xae\xc5\x0b\t\xb2a\x1d\x06\xd4E3;\xcaR^\x10E\x9c\x8e[\x1c\xea\xa7\xe3\xde\xb7\x8f\x9c\xd4\x9d@\xe3}MCl\x89x\xdb\xc2A\x9f\xbdT\xd7\x9f\x7fJ\xd3\xe4\x1fbu\xc1\xe9.\x08\xcb\x9a}\x9aq\x94\x0fOy\xe1\xd9\xa6\x08\xed\xc0k\x114\x08{\xa5\xa5`\xa0c\x84\xec\x8fX\x9f3K7\x9da+E^\x9d\xa5\xce\x8d\xa3S\xc0\xac\x010\xfa{Ww\x9bA\xdb\xc9x\xc1=\x04\xc0'
049e00e9775dda55f9368c3e84de30071257d536427dc6bfd2de888473a7914c3bb33504c598a5b33a172f400a4590f41fb8b5da18f5804d205e8aba2f692facacb4787713de93dfaa218e952ef5c368dcee97678cc587996dcaf57644f7b89d9d6d633e83d9a15e5d088

In [30]:
embed_lsb("lsb_w_ECC.png", ciphertext.hex())

In [31]:
extract_lsb("lsb_w_ECC.png")

'049e00e9775dda55f9368c3e84de30071257d536427dc6bfd2de888473a7914c3bb33504c598a5b33a172f400a4590f41fb8b5da18f5804d205e8aba2f692facacb4787713de93dfaa218e952ef5c368dcee97678cc587996dcaf57644f7b89d9d6d633e83d9a15e5d0888774d3b4fe3e620b66ece887ae5df70beeba5894fc4b0674f51e8574671e747b9b8d7ef5a73e386e2e7cb1f7ba15fd8ea03aec50b09b2611d06d445333bca525e10459c8e5b1ceaa7e3deb78f9cd49d40e37d4d436c8978dbc2419fbd54d79f7f4ad3e41f6275c1e92e08cb9a7d9a71940f4f79e1d9a608edc06b1134087ba5a560a06384ec8f589f334b379d612b455e9da5ce8da353c0ac0130fa7b57779b41dbc978c13d04c0'

In [32]:
calculate_psnr_2("lsb_w_ECC.png")

PSNR: 73.71881716622424


73.71881716622424

## Other LSB Package

In [33]:
from ecies.utils import generate_eth_key
from ecies import encrypt, decrypt

private_key = generate_eth_key()
public_key = private_key.public_key

public_key_hex = public_key.to_hex()
private_key_hex = private_key.to_hex()

print("Public Key:", public_key_hex)
print("Private Key:", private_key_hex)

ciphertext = encrypt(public_key_hex, CONTENT.encode())

print("Ciphertext:", ciphertext.hex())

decrypted_message = decrypt(private_key_hex, ciphertext)
print("Decrypted Message:", decrypted_message.decode())

Public Key: 0xca7f39a10fabdce85c8934987571a556392b9e20ba12e45115408b013499672511f9fadfdd635b3a7cd95cfb931b21e2b75765dec1ed55eedd842394bfa6a738
Private Key: 0x84493dc0bf1c66198a3f8b95a5c8c620388078949e2d5260afabddd90a107dc9
Ciphertext: 044ff0124b6dc6f03895b6880eeef1fe3ae10bef700b30f7b5c2ca613d596e7c1868d25c01ad9db3cf3ba1c40b6789867ab3430aab1d1abbeba1b355104543cb3d9317efc1be8355b5e3dbcb7da21943191300c1d04243e7a32e17d39315180caa982b5045298a87daedef79c55e8fe46afa841514eb9db5ecb1b9b3b44e406f1ed3a9f05cdc3d7d0ef08269f45520cd3f34df19e80fa3b10249ae1067e8e9bc2c0f3fbf31e106e1b7c187afd97007e6ab970ee582c9ac055ac16d2b8557bdb6c83aa7a96563b9ed624e24eed19830b84178fd0e642fd35f546ab536ccd329ffc8cddbf30fbffaa5fb150a9e0ce02552b22a979e855e24884385f470e65aa72fdfc75013d0892512dabe17cafd3687a79cea
Decrypted Message: 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup


In [34]:
print(ciphertext)
print(ciphertext.hex())

b'\x04O\xf0\x12Km\xc6\xf08\x95\xb6\x88\x0e\xee\xf1\xfe:\xe1\x0b\xefp\x0b0\xf7\xb5\xc2\xcaa=Yn|\x18h\xd2\\\x01\xad\x9d\xb3\xcf;\xa1\xc4\x0bg\x89\x86z\xb3C\n\xab\x1d\x1a\xbb\xeb\xa1\xb3U\x10EC\xcb=\x93\x17\xef\xc1\xbe\x83U\xb5\xe3\xdb\xcb}\xa2\x19C\x19\x13\x00\xc1\xd0BC\xe7\xa3.\x17\xd3\x93\x15\x18\x0c\xaa\x98+PE)\x8a\x87\xda\xed\xefy\xc5^\x8f\xe4j\xfa\x84\x15\x14\xeb\x9d\xb5\xec\xb1\xb9\xb3\xb4N@o\x1e\xd3\xa9\xf0\\\xdc=}\x0e\xf0\x82i\xf4U \xcd?4\xdf\x19\xe8\x0f\xa3\xb1\x02I\xae\x10g\xe8\xe9\xbc,\x0f?\xbf1\xe1\x06\xe1\xb7\xc1\x87\xaf\xd9p\x07\xe6\xab\x97\x0e\xe5\x82\xc9\xac\x05Z\xc1m+\x85W\xbd\xb6\xc8:\xa7\xa9ec\xb9\xedbN$\xee\xd1\x980\xb8Ax\xfd\x0ed/\xd3_Tj\xb56\xcc\xd3)\xff\xc8\xcd\xdb\xf3\x0f\xbf\xfa\xa5\xfb\x15\n\x9e\x0c\xe0%R\xb2*\x97\x9e\x85^$\x88C\x85\xf4p\xe6Z\xa7/\xdf\xc7P\x13\xd0\x89%\x12\xda\xbe\x17\xca\xfd6\x87\xa7\x9c\xea'
044ff0124b6dc6f03895b6880eeef1fe3ae10bef700b30f7b5c2ca613d596e7c1868d25c01ad9db3cf3ba1c40b6789867ab3430aab1d1abbeba1b355104543cb3d9317efc1be8355b5e3dbcb7d

In [37]:
secret = lsb.hide("/content/ori-image/lena.png", ciphertext.hex())

secret.save("/content/stego-image/lsb_2_w_ECC.png")

In [38]:
print(lsb.reveal("/content/stego-image/lsb_2_w_ECC.png"))

044ff0124b6dc6f03895b6880eeef1fe3ae10bef700b30f7b5c2ca613d596e7c1868d25c01ad9db3cf3ba1c40b6789867ab3430aab1d1abbeba1b355104543cb3d9317efc1be8355b5e3dbcb7da21943191300c1d04243e7a32e17d39315180caa982b5045298a87daedef79c55e8fe46afa841514eb9db5ecb1b9b3b44e406f1ed3a9f05cdc3d7d0ef08269f45520cd3f34df19e80fa3b10249ae1067e8e9bc2c0f3fbf31e106e1b7c187afd97007e6ab970ee582c9ac055ac16d2b8557bdb6c83aa7a96563b9ed624e24eed19830b84178fd0e642fd35f546ab536ccd329ffc8cddbf30fbffaa5fb150a9e0ce02552b22a979e855e24884385f470e65aa72fdfc75013d0892512dabe17cafd3687a79cea


In [39]:
calculate_psnr_2("lsb_2_w_ECC.png")

PSNR: 73.51378506924091


73.51378506924091

In [40]:
import numpy as np
import cv2

def calculate_psnr(original_image, modified_image):
    mse = np.mean((original_image - modified_image) ** 2)
    if mse == 0:
        return 100  # Perfect match
    print(mse)
    pixel_max = 255.0
    return 20 * np.log10(pixel_max / np.sqrt(mse))

# Example usage
original_image = cv2.imread('/content/ori-image/lena.png')  # Load the original image
modified_image = cv2.imread('/content/stego-image/lsb_2_w_ECC.png')  # Load the modified image after LSB embedding

psnr_value = calculate_psnr(original_image, modified_image)
print(f"PSNR value: {psnr_value}")


0.002895355224609375
PSNR value: 73.51378506924091


# LSB with LZW

## LZW

In [41]:
import math

def lzw_compress(uncompressed):
    """Compress the string using LZW algorithm."""
    dict_size = 256
    dictionary = {chr(i): i for i in range(dict_size)}

    w = ""
    compressed_data = []

    for c in uncompressed:
        wc = w + c
        if wc in dictionary:
            w = wc
        else:
            compressed_data.append(dictionary[w])
            dictionary[wc] = dict_size
            dict_size += 1
            w = c

    if w:
        compressed_data.append(dictionary[w])

    return compressed_data, dict_size

def lzw_decompress(compressed):
    """Decompress the LZW compressed data."""
    dict_size = 256
    dictionary = {i: chr(i) for i in range(dict_size)}

    w = chr(compressed.pop(0))
    decompressed_data = w

    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError(f"Bad compressed k: {k} (dictionary size: {dict_size})")

        decompressed_data += entry
        dictionary[dict_size] = w + entry[0]
        dict_size += 1
        w = entry

    return decompressed_data

# Function to calculate size in bytes or bits
def calculate_size(data, dict_size=None):
    if isinstance(data, str):
        # Size in bytes and bits for string (1 byte = 8 bits for each character)
        size_in_bytes = len(data.encode('utf-8'))
        size_in_bits = size_in_bytes * 8
        return size_in_bytes, size_in_bits  # Size in bytes and bits
    elif isinstance(data, list):
        # Calculate the bit size for compressed data (variable bit-length encoding)
        if dict_size is None:
            raise ValueError("Dictionary size needed for proper bit calculation!")
        bits_needed = math.ceil(math.log2(dict_size))  # Number of bits required
        total_bits = len(data) * bits_needed
        return total_bits // 8, total_bits  # Returns size in bytes and bits

# Example usage:
input_string = CONTENT
print(f"Original String: {input_string}")

# Compress the input string
compressed, dict_size = lzw_compress(input_string)
decompressed = lzw_decompress(compressed)

# Sizes
original_size_bytes, original_size_bits = calculate_size(input_string)  # Original size in bytes and bits
compressed_size_bytes, compressed_size_bits = calculate_size(compressed, dict_size)  # Compressed size in bytes and bits
decompressed_size_bytes, decompressed_size_bits = calculate_size(decompressed)  # Decompressed size in bytes and bits

print(f"\nOriginal String Size: {original_size_bytes} bytes ({original_size_bits} bits)")
print(f"Compressed Data Size: {compressed_size_bytes} bytes ({compressed_size_bits} bits)")
print(f"Decompressed String Size: {decompressed_size_bytes} bytes ({decompressed_size_bits} bits)")

# Check if decompressed string matches the original string
if decompressed == input_string:
    print("Decompression was successful!")
    print()
    print(f"Decompressed String: {decompressed}")
else:
    print("Decompression failed.")

Original String: 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup

Original String Size: 177 bytes (1416 bits)
Compressed Data Size: 166 bytes (1332 bits)
Decompressed String Size: 177 bytes (1416 bits)
Decompression was successful!

Decompressed String: 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup


In [42]:
print(compressed)

[51, 52, 56, 54, 49, 48, 52, 48, 49, 57, 55, 48, 268, 53, 35, 99, 104, 114, 105, 115, 116, 111, 102, 101, 114, 42, 100, 280, 105, 97, 110, 42, 98, 117, 100, 285, 110, 277, 35, 116, 101, 103, 97, 108, 35, 265, 266, 45, 48, 51, 304, 52, 35, 108, 97, 107, 105, 45, 310, 312, 35, 98, 35, 106, 108, 46, 42, 112, 299, 97, 42, 50, 50, 42, 110, 111, 322, 51, 48, 35, 269, 336, 49, 55, 35, 109, 101, 106, 97, 115, 101, 109, 42, 296, 110, 298, 104, 35, 107, 114, 97, 109, 97, 116, 354, 359, 104, 111, 108, 105, 107, 317, 101, 108, 117, 348, 107, 97, 119, 105, 110, 35, 112, 369, 97, 344, 114, 47, 358, 104, 345, 275, 119, 97, 35, 119, 110, 105, 35, 346, 371, 117, 281, 104, 105, 100, 117, 112]


In [43]:
compressed_str = ','.join(map(str, compressed))

compressed_str

'51,52,56,54,49,48,52,48,49,57,55,48,268,53,35,99,104,114,105,115,116,111,102,101,114,42,100,280,105,97,110,42,98,117,100,285,110,277,35,116,101,103,97,108,35,265,266,45,48,51,304,52,35,108,97,107,105,45,310,312,35,98,35,106,108,46,42,112,299,97,42,50,50,42,110,111,322,51,48,35,269,336,49,55,35,109,101,106,97,115,101,109,42,296,110,298,104,35,107,114,97,109,97,116,354,359,104,111,108,105,107,317,101,108,117,348,107,97,119,105,110,35,112,369,97,344,114,47,358,104,345,275,119,97,35,119,110,105,35,346,371,117,281,104,105,100,117,112'

In [44]:
embed_lsb("lsb_w_LZW.png", compressed_str)

In [45]:
extracted_lzw_from_lsb = extract_lsb("lsb_w_LZW.png")
extracted_lzw_from_lsb

'51,52,56,54,49,48,52,48,49,57,55,48,268,53,35,99,104,114,105,115,116,111,102,101,114,42,100,280,105,97,110,42,98,117,100,285,110,277,35,116,101,103,97,108,35,265,266,45,48,51,304,52,35,108,97,107,105,45,310,312,35,98,35,106,108,46,42,112,299,97,42,50,50,42,110,111,322,51,48,35,269,336,49,55,35,109,101,106,97,115,101,109,42,296,110,298,104,35,107,114,97,109,97,116,354,359,104,111,108,105,107,317,101,108,117,348,107,97,119,105,110,35,112,369,97,344,114,47,358,104,345,275,119,97,35,119,110,105,35,346,371,117,281,104,105,100,117,112'

In [46]:
calculate_psnr_2("lsb_w_LZW.png")

PSNR: 73.77314241018492


73.77314241018492

In [47]:
extracted_compressed_data = list(map(int, extracted_lzw_from_lsb.split(',')))
extracted_compressed_data

[51,
 52,
 56,
 54,
 49,
 48,
 52,
 48,
 49,
 57,
 55,
 48,
 268,
 53,
 35,
 99,
 104,
 114,
 105,
 115,
 116,
 111,
 102,
 101,
 114,
 42,
 100,
 280,
 105,
 97,
 110,
 42,
 98,
 117,
 100,
 285,
 110,
 277,
 35,
 116,
 101,
 103,
 97,
 108,
 35,
 265,
 266,
 45,
 48,
 51,
 304,
 52,
 35,
 108,
 97,
 107,
 105,
 45,
 310,
 312,
 35,
 98,
 35,
 106,
 108,
 46,
 42,
 112,
 299,
 97,
 42,
 50,
 50,
 42,
 110,
 111,
 322,
 51,
 48,
 35,
 269,
 336,
 49,
 55,
 35,
 109,
 101,
 106,
 97,
 115,
 101,
 109,
 42,
 296,
 110,
 298,
 104,
 35,
 107,
 114,
 97,
 109,
 97,
 116,
 354,
 359,
 104,
 111,
 108,
 105,
 107,
 317,
 101,
 108,
 117,
 348,
 107,
 97,
 119,
 105,
 110,
 35,
 112,
 369,
 97,
 344,
 114,
 47,
 358,
 104,
 345,
 275,
 119,
 97,
 35,
 119,
 110,
 105,
 35,
 346,
 371,
 117,
 281,
 104,
 105,
 100,
 117,
 112]

In [48]:
# decompressed = lzw_decompress(extracted_compressed_data)
decompressed

'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup'

only decompress sequenced after compress

## LZW Package

In [49]:
!git clone https://github.com/joeatwork/python-lzw

Cloning into 'python-lzw'...
remote: Enumerating objects: 80, done.
remote: Total 80 (delta 0), reused 0 (delta 0), pack-reused 80 (from 1)
Receiving objects: 100% (80/80), 3.22 MiB | 15.36 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [50]:
!cd /content/python-lzw && mv lzw/ /content/ && python setup.py install

running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https

In [51]:
import lzw

# Example text to compress (can be any string)
original_text = CONTENT

# Convert the original text into bytes (UTF-8 encoding)
original_bytes = original_text.encode('utf-8')

# Part 1: Compression (in-memory)
compressed = lzw.compress(original_bytes)

# Convert compressed generator to a list to see its contents
compressed_list = list(compressed)

# Part 2: Decompression (in-memory)
decompressed_chunks = lzw.decompress(compressed_list)

# Join the decompressed chunks into a single byte string
decompressed_bytes = b"".join(decompressed_chunks)

# Convert decompressed bytes back to a string
decompressed_text = decompressed_bytes.decode('utf-8')

# Print each variable
print("Original Text (String):", original_text)
print("Original Bytes:", original_bytes)
print("Compressed List (Byte Chunks):", compressed_list)
print("Decompressed Bytes:", decompressed_bytes)
print("Decompressed Text (String):", decompressed_text)

# Part 3: Print sizes and results for comparison
original_size = len(original_bytes)
compressed_size = len(b"".join(compressed_list))  # Join to get compressed byte size
decompressed_size = len(decompressed_bytes)

# Print the sizes in bytes
print(f"\nOriginal size: {original_size} bytes")
print(f"Compressed size: {compressed_size} bytes")
print(f"Decompressed size: {decompressed_size} bytes")

# Check if decompression was successful
if decompressed_text == original_text:
    print("Decompression was successful!")
else:
    print("Decompression failed.")

Original Text (String): 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup
Original Bytes: b'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup'
Compressed List (Byte Chunks): [b'\x19', b'\x8c', b'\xc6', b'\x83', b'\x81', b'\xb0', b'\xc4', b'`', b'4', b'\x18', b'\x0c', b'G', b'#', b'q', b'\x84', b'8', b'j', b'#', b'1', b'\x9a', b'\x0e', b'F', b'\x93', b'\x99', b'\xd0', b'\xde', b'f', b'2', b'\x9c', b'\x85', b'F', b'H', b'\xd1', b'\xa4', b'\xc2', b'n', b'\x15', b'\x18', b'\x8e', b'\xa6', b'H', b'\xf9', b'\xba', b'.', b'#', b':', b'\x19', b'L', b'\xe6', b'\x13', b'`', b'\x8e', b'\x17', b'\x0c', b'\x16', b'\x8c', b'\x06', b's', b'!', b'\xa0', b'\x8c', b'\xd8', b'a', b'5', b'\x9a', b'E', b'\xb3', b'\x89', b'\xd0', b'\x8c', b'\xc4', b'#', b'

In [52]:
compressed

<generator object BitPacker.pack at 0x7967435823b0>

In [53]:
compressed_list

[b'\x19',
 b'\x8c',
 b'\xc6',
 b'\x83',
 b'\x81',
 b'\xb0',
 b'\xc4',
 b'`',
 b'4',
 b'\x18',
 b'\x0c',
 b'G',
 b'#',
 b'q',
 b'\x84',
 b'8',
 b'j',
 b'#',
 b'1',
 b'\x9a',
 b'\x0e',
 b'F',
 b'\x93',
 b'\x99',
 b'\xd0',
 b'\xde',
 b'f',
 b'2',
 b'\x9c',
 b'\x85',
 b'F',
 b'H',
 b'\xd1',
 b'\xa4',
 b'\xc2',
 b'n',
 b'\x15',
 b'\x18',
 b'\x8e',
 b'\xa6',
 b'H',
 b'\xf9',
 b'\xba',
 b'.',
 b'#',
 b':',
 b'\x19',
 b'L',
 b'\xe6',
 b'\x13',
 b'`',
 b'\x8e',
 b'\x17',
 b'\x0c',
 b'\x16',
 b'\x8c',
 b'\x06',
 b's',
 b'!',
 b'\xa0',
 b'\x8c',
 b'\xd8',
 b'a',
 b'5',
 b'\x9a',
 b'E',
 b'\xb3',
 b'\x89',
 b'\xd0',
 b'\x8c',
 b'\xc4',
 b'#',
 b'5',
 b'\x1b',
 b'\x05',
 b'\xc2',
 b'\xa3',
 b'\x84',
 b'\xb4',
 b'\xc2',
 b'*',
 b'\x19',
 b'\x0c',
 b'\x85',
 b'F',
 b'\xe3',
 b'}',
 b'\x10',
 b'f',
 b'0',
 b'\x11',
 b'\xc3',
 b'\xea',
 b'C',
 b'\x11',
 b'\xb8',
 b'\x8c',
 b'\xda',
 b'e',
 b'5',
 b'\x18',
 b'N',
 b'f',
 b'S',
 b'h',
 b'\xaa',
 b'T',
 b'n',
 b'\x96',
 b'\x1a',
 b'\x04',
 b'f',
 b'\xb3',

In [54]:
type(compressed_list)

list

In [55]:
embed_lsb("lsb_w_LZW_package.png", compressed_list)

In [56]:
extracted_data = extract_lsb("lsb_w_LZW_package.png")
extracted_data

'\x19\x8cÆ\x83\x81°Ä`4\x18\x0cG#q\x848j#1\x9a\x0eF\x93\x99ÐÞf2\x9c\x85FHÑ¤Ân\x15\x18\x8e¦Hùº.#:\x19Læ\x13`\x8e\x17\x0c\x16\x8c\x06s!\xa0\x8cØa5\x9aE³\x89Ð\x8cÄ#5\x1b\x05Â£\x84´Â*\x19\x0c\x85Fã}\x10f0\x11ÃêC\x11¸\x8cÚe5\x18NfShªTn\x96\x1a\x04f³\x91\x84Úa:Y-&\x83y°Òk\x9f\x99M\x87Zñ¬Âw4\x9b\x84g\x0b\x99\x86´r\x17Ú\r\x15¸©ÜÂ#;\x9b\x8d":åÔë\x1b4\x1aL\x87S\x84\x00'

In [57]:
output_list = [repr(byte.encode('latin1')) for byte in extracted_data]

# Print in the requested format
formatted_output = "[\n" + ",\n ".join(output_list) + "\n]"

print(formatted_output)

[
b'\x19',
 b'\x8c',
 b'\xc6',
 b'\x83',
 b'\x81',
 b'\xb0',
 b'\xc4',
 b'`',
 b'4',
 b'\x18',
 b'\x0c',
 b'G',
 b'#',
 b'q',
 b'\x84',
 b'8',
 b'j',
 b'#',
 b'1',
 b'\x9a',
 b'\x0e',
 b'F',
 b'\x93',
 b'\x99',
 b'\xd0',
 b'\xde',
 b'f',
 b'2',
 b'\x9c',
 b'\x85',
 b'F',
 b'H',
 b'\xd1',
 b'\xa4',
 b'\xc2',
 b'n',
 b'\x15',
 b'\x18',
 b'\x8e',
 b'\xa6',
 b'H',
 b'\xf9',
 b'\xba',
 b'.',
 b'#',
 b':',
 b'\x19',
 b'L',
 b'\xe6',
 b'\x13',
 b'`',
 b'\x8e',
 b'\x17',
 b'\x0c',
 b'\x16',
 b'\x8c',
 b'\x06',
 b's',
 b'!',
 b'\xa0',
 b'\x8c',
 b'\xd8',
 b'a',
 b'5',
 b'\x9a',
 b'E',
 b'\xb3',
 b'\x89',
 b'\xd0',
 b'\x8c',
 b'\xc4',
 b'#',
 b'5',
 b'\x1b',
 b'\x05',
 b'\xc2',
 b'\xa3',
 b'\x84',
 b'\xb4',
 b'\xc2',
 b'*',
 b'\x19',
 b'\x0c',
 b'\x85',
 b'F',
 b'\xe3',
 b'}',
 b'\x10',
 b'f',
 b'0',
 b'\x11',
 b'\xc3',
 b'\xea',
 b'C',
 b'\x11',
 b'\xb8',
 b'\x8c',
 b'\xda',
 b'e',
 b'5',
 b'\x18',
 b'N',
 b'f',
 b'S',
 b'h',
 b'\xaa',
 b'T',
 b'n',
 b'\x96',
 b'\x1a',
 b'\x04',
 b'f',
 b'\xb3'

In [58]:
type(formatted_output)

str

In [59]:
byte_list = [bytes([ord(char)]) for char in extracted_data]

# Print the byte list in the required format
print(byte_list)

[b'\x19', b'\x8c', b'\xc6', b'\x83', b'\x81', b'\xb0', b'\xc4', b'`', b'4', b'\x18', b'\x0c', b'G', b'#', b'q', b'\x84', b'8', b'j', b'#', b'1', b'\x9a', b'\x0e', b'F', b'\x93', b'\x99', b'\xd0', b'\xde', b'f', b'2', b'\x9c', b'\x85', b'F', b'H', b'\xd1', b'\xa4', b'\xc2', b'n', b'\x15', b'\x18', b'\x8e', b'\xa6', b'H', b'\xf9', b'\xba', b'.', b'#', b':', b'\x19', b'L', b'\xe6', b'\x13', b'`', b'\x8e', b'\x17', b'\x0c', b'\x16', b'\x8c', b'\x06', b's', b'!', b'\xa0', b'\x8c', b'\xd8', b'a', b'5', b'\x9a', b'E', b'\xb3', b'\x89', b'\xd0', b'\x8c', b'\xc4', b'#', b'5', b'\x1b', b'\x05', b'\xc2', b'\xa3', b'\x84', b'\xb4', b'\xc2', b'*', b'\x19', b'\x0c', b'\x85', b'F', b'\xe3', b'}', b'\x10', b'f', b'0', b'\x11', b'\xc3', b'\xea', b'C', b'\x11', b'\xb8', b'\x8c', b'\xda', b'e', b'5', b'\x18', b'N', b'f', b'S', b'h', b'\xaa', b'T', b'n', b'\x96', b'\x1a', b'\x04', b'f', b'\xb3', b'\x91', b'\x84', b'\xda', b'a', b':', b'Y', b'-', b'&', b'\x83', b'y', b'\xb0', b'\xd2', b'k', b'\x9f', b'\x99

In [60]:
type(byte_list)

list

In [61]:
calculate_psnr_2("lsb_w_LZW_package.png")

PSNR: 78.72417421832482


78.72417421832482

In [62]:
# Part 2: Decompression (in-memory)
decompressed_chunks = lzw.decompress(byte_list)

# Join the decompressed chunks into a single byte string
decompressed_bytes = b"".join(decompressed_chunks)

# Convert decompressed bytes back to a string
decompressed_text = decompressed_bytes.decode('utf-8')

print("Decompressed Text (String):", decompressed_text)

Decompressed Text (String): 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup


# LSB, ECC, with LZW

In [63]:
CONTENT

'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup'

In [64]:
from ecies.utils import generate_eth_key
from ecies import encrypt, decrypt

private_key = generate_eth_key()
public_key = private_key.public_key

public_key_hex = public_key.to_hex()
private_key_hex = private_key.to_hex()

print("Public Key:", public_key_hex)
print("Private Key:", private_key_hex)

ciphertext = encrypt(public_key_hex, CONTENT.encode())

print("Ciphertext:", ciphertext.hex())

decrypted_message = decrypt(private_key_hex, ciphertext)
print("Decrypted Message:", decrypted_message.decode())

Public Key: 0xc9c55c4a06ea9f6c4348bf22b932a0b15c4e4031b060fd72d002d974eca31b09bab7acdfdfafd92cdb13a349a23cabcc4a243392b77a62763d71eab022a87aae
Private Key: 0x865ce09b928f829b588e0e098ba34e2b5884d39f43d4765200e2414e1d37d3f1
Ciphertext: 045e270f58e50d3e481da10129c36ff059c49468eccde52a351cf27ec30b1c3942f25a42256daeeb3c6a3c9082d1fb70c9510451246f4e605e3cf1fb7ceceeb2e08b4eb945ef9a643a85ea46a8f0a3f418e754b7202f97be3d4ab479548f59c8db9ee80a650d206f4b44aef2355c281bef9bb7a77d7f7be3860610788af369c6c970a4f8c0b22eeac519a3017286e07d1549df3dc65e163d3ffc7bbd3cfa829e95c0e32d39e8757fc971243f3f7dcd97fb3179d285aaa7150a01628d3689c8c632c0df923634dc16a36acc50bb7f3dedd16b6f58d3fa00080443e7d111bd904092fbeb49d6bcaf23615c746796f8497315d6c673b91e01f8e427800e2342afa8ade6c7442617150e761894dd83cf21d28bcb
Decrypted Message: 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup


In [65]:
ciphertext

b"\x04^'\x0fX\xe5\r>H\x1d\xa1\x01)\xc3o\xf0Y\xc4\x94h\xec\xcd\xe5*5\x1c\xf2~\xc3\x0b\x1c9B\xf2ZB%m\xae\xeb<j<\x90\x82\xd1\xfbp\xc9Q\x04Q$oN`^<\xf1\xfb|\xec\xee\xb2\xe0\x8bN\xb9E\xef\x9ad:\x85\xeaF\xa8\xf0\xa3\xf4\x18\xe7T\xb7 /\x97\xbe=J\xb4yT\x8fY\xc8\xdb\x9e\xe8\ne\r oKD\xae\xf25\\(\x1b\xef\x9b\xb7\xa7}\x7f{\xe3\x86\x06\x10x\x8a\xf3i\xc6\xc9p\xa4\xf8\xc0\xb2.\xea\xc5\x19\xa3\x01r\x86\xe0}\x15I\xdf=\xc6^\x16=?\xfc{\xbd<\xfa\x82\x9e\x95\xc0\xe3-9\xe8u\x7f\xc9q$??}\xcd\x97\xfb1y\xd2\x85\xaa\xa7\x15\n\x01b\x8d6\x89\xc8\xc62\xc0\xdf\x9264\xdc\x16\xa3j\xccP\xbb\x7f=\xed\xd1koX\xd3\xfa\x00\x08\x04C\xe7\xd1\x11\xbd\x90@\x92\xfb\xebI\xd6\xbc\xaf#a\\tg\x96\xf8Is\x15\xd6\xc6s\xb9\x1e\x01\xf8\xe4'\x80\x0e#B\xaf\xa8\xad\xe6\xc7D&\x17\x15\x0ev\x18\x94\xdd\x83\xcf!\xd2\x8b\xcb"

In [66]:
import lzw

# Example text to compress (can be any string)
original_text = ciphertext.hex()

# Convert the original text into bytes (UTF-8 encoding)
original_bytes = original_text.encode('utf-8')

# Part 1: Compression (in-memory)
compressed = lzw.compress(original_bytes)

# Convert compressed generator to a list to see its contents
compressed_list = list(compressed)

# Part 2: Decompression (in-memory)
decompressed_chunks = lzw.decompress(compressed_list)

# Join the decompressed chunks into a single byte string
decompressed_bytes = b"".join(decompressed_chunks)

# Convert decompressed bytes back to a string
decompressed_text = decompressed_bytes.decode('utf-8')

# Print each variable
print("Original Text (String):", original_text)
print("Original Bytes:", original_bytes)
print("Compressed List (Byte Chunks):", compressed_list)
print("Decompressed Bytes:", decompressed_bytes)
print("Decompressed Text (String):", decompressed_text)

# Part 3: Print sizes and results for comparison
original_size = len(original_bytes)
compressed_size = len(b"".join(compressed_list))  # Join to get compressed byte size
decompressed_size = len(decompressed_bytes)

# Print the sizes in bytes
print(f"\nOriginal size: {original_size} bytes")
print(f"Compressed size: {compressed_size} bytes")
print(f"Decompressed size: {decompressed_size} bytes")

# Check if decompression was successful
if decompressed_text == original_text:
    print("Decompression was successful!")
else:
    print("Decompression failed.")

Original Text (String): 045e270f58e50d3e481da10129c36ff059c49468eccde52a351cf27ec30b1c3942f25a42256daeeb3c6a3c9082d1fb70c9510451246f4e605e3cf1fb7ceceeb2e08b4eb945ef9a643a85ea46a8f0a3f418e754b7202f97be3d4ab479548f59c8db9ee80a650d206f4b44aef2355c281bef9bb7a77d7f7be3860610788af369c6c970a4f8c0b22eeac519a3017286e07d1549df3dc65e163d3ffc7bbd3cfa829e95c0e32d39e8757fc971243f3f7dcd97fb3179d285aaa7150a01628d3689c8c632c0df923634dc16a36acc50bb7f3dedd16b6f58d3fa00080443e7d111bd904092fbeb49d6bcaf23615c746796f8497315d6c673b91e01f8e427800e2342afa8ade6c7442617150e761894dd83cf21d28bcb
Original Bytes: b'045e270f58e50d3e481da10129c36ff059c49468eccde52a351cf27ec30b1c3942f25a42256daeeb3c6a3c9082d1fb70c9510451246f4e605e3cf1fb7ceceeb2e08b4eb945ef9a643a85ea46a8f0a3f418e754b7202f97be3d4ab479548f59c8db9ee80a650d206f4b44aef2355c281bef9bb7a77d7f7be3860610788af369c6c970a4f8c0b22eeac519a3017286e07d1549df3dc65e163d3ffc7bbd3cfa829e95c0e32d39e8757fc971243f3f7dcd97fb3179d285aaa7150a01628d3689c8c632c0df923634dc16a36acc50b

In [67]:
embed_lsb("lsb_w_ECC_LZW_package.png", compressed_list)

In [68]:
extracted_data = extract_lsb("lsb_w_ECC_LZW_package.png")
extracted_data

'\x18\r\x06¦Q\x90Ü`f\x1a\x8e\x0c£Q\x81\x90fe\x1a\x0e\x06&C\x08Ä`1\x19\x0eLc1±\x98Ì0\x1aÆÆ\x83\x91\xa0Ú\x16c1\x99!\x83#\x08Ìj11\x99\xa0ÆXàÀÅ1\x19ÉFS1©\x84h2\x19\rFÑS)\x94Ä31\x8d¥Æ1ÈÀp22\x0cLÆ(=6a\x02\x98\x0c¤Æa¡\x94m!2ÒLÕ:©\x8ekF1\x0c\x8c´ó\x15xÅ%\x82\x19\x87&\x11°Ðfa\x1cA\'ô±Ä\x82]]\x18ÂÆãQ¥Td0\x9e\x0eFæ+\x11\x90ha·\rÇ8[ô\x88Æ82\\(Ã\x81\x85Ö\x1adÄG°ÃL\x81\x96g/\x1a\x98ÆQ<mÌÅU0\x8dÆæA¹\x9b\x19b\x1cX\x06Ñq¸àpa3Gccjl\x1e~f\x1c\x18æô\x1a1\x84Ç0º\x0cã\x03}`ÚÙ´\x18áG&N\x11\x92\x95\x04\x18\x8d\x86pøù\x8f\x18b\x87Ì¯1£.S\x97b¨Î\x8c£\x8c&Û\x8d\x19»ð¶ýó&-T\x0cÃ\x16M¡^\x86\x18\x187v\x99ä`6k\x10ô\x9d\x1br\x9e0ÉËwC\x90É\x1d\x0cÃG}âK\x94´¥\rl\x1bg\x90e\x19\x15 ØbG\x90§\x94a\x0c"Öu¥D\x1d\x90Ä1zTàÐ0\x85\x95E\x1d$\x19"q\x8dÁ\x85ÛÆ¨7I\x994x8I\x03x\x045\x8fT©)p\x0cVÅM\x0bP\x1bèµ\x05\x86RÑ\x99y\x18R·\x16DP\x1bÈ!\r\x19Cvð8IbPác\x0c\x91F°b\x18Æ$\x00'

In [69]:
calculate_psnr_2("lsb_w_ECC_LZW_package.png")

PSNR: 74.67444150429242


74.67444150429242

In [70]:
byte_list = [bytes([ord(char)]) for char in extracted_data]

# Print the byte list in the required format
print(byte_list)

[b'\x18', b'\r', b'\x06', b'\xa6', b'Q', b'\x90', b'\xdc', b'`', b'f', b'\x1a', b'\x8e', b'\x0c', b'\xa3', b'Q', b'\x81', b'\x90', b'f', b'e', b'\x1a', b'\x0e', b'\x06', b'&', b'C', b'\x08', b'\xc4', b'`', b'1', b'\x19', b'\x0e', b'L', b'c', b'1', b'\xb1', b'\x98', b'\xcc', b'0', b'\x1a', b'\xc6', b'\xc6', b'\x83', b'\x91', b'\xa0', b'\xda', b'\x16', b'c', b'1', b'\x99', b'!', b'\x83', b'#', b'\x08', b'\xcc', b'j', b'1', b'1', b'\x99', b'\xa0', b'\xc6', b'X', b'\xe0', b'\xc0', b'\xc5', b'1', b'\x19', b'\xc9', b'F', b'S', b'1', b'\xa9', b'\x84', b'h', b'2', b'\x19', b'\r', b'F', b'\xd1', b'S', b')', b'\x94', b'\xc4', b'3', b'1', b'\x8d', b'\xa5', b'\xc6', b'1', b'\xc8', b'\xc0', b'p', b'2', b'2', b'\x0c', b'L', b'\xc6', b'(', b'=', b'6', b'a', b'\x02', b'\x98', b'\x0c', b'\xa4', b'\xc6', b'a', b'\xa1', b'\x94', b'm', b'!', b'2', b'\xd2', b'L', b'\xd5', b':', b'\xa9', b'\x8e', b'k', b'F', b'1', b'\x0c', b'\x8c', b'\xb4', b'\xf3', b'\x15', b'x', b'\xc5', b'%', b'\x82', b'\x19', b'\x87', b

In [71]:
# Part 2: Decompression (in-memory)
decompressed_chunks = lzw.decompress(byte_list)

# Join the decompressed chunks into a single byte string
decompressed_bytes = b"".join(decompressed_chunks)

# Convert decompressed bytes back to a string
decompressed_text = decompressed_bytes.decode('utf-8')

print("Decompressed Text (String):", decompressed_text)

Decompressed Text (String): 045e270f58e50d3e481da10129c36ff059c49468eccde52a351cf27ec30b1c3942f25a42256daeeb3c6a3c9082d1fb70c9510451246f4e605e3cf1fb7ceceeb2e08b4eb945ef9a643a85ea46a8f0a3f418e754b7202f97be3d4ab479548f59c8db9ee80a650d206f4b44aef2355c281bef9bb7a77d7f7be3860610788af369c6c970a4f8c0b22eeac519a3017286e07d1549df3dc65e163d3ffc7bbd3cfa829e95c0e32d39e8757fc971243f3f7dcd97fb3179d285aaa7150a01628d3689c8c632c0df923634dc16a36acc50bb7f3dedd16b6f58d3fa00080443e7d111bd904092fbeb49d6bcaf23615c746796f8497315d6c673b91e01f8e427800e2342afa8ade6c7442617150e761894dd83cf21d28bcb


In [72]:
byte_data = decompressed_text.encode()
byte_data

b'045e270f58e50d3e481da10129c36ff059c49468eccde52a351cf27ec30b1c3942f25a42256daeeb3c6a3c9082d1fb70c9510451246f4e605e3cf1fb7ceceeb2e08b4eb945ef9a643a85ea46a8f0a3f418e754b7202f97be3d4ab479548f59c8db9ee80a650d206f4b44aef2355c281bef9bb7a77d7f7be3860610788af369c6c970a4f8c0b22eeac519a3017286e07d1549df3dc65e163d3ffc7bbd3cfa829e95c0e32d39e8757fc971243f3f7dcd97fb3179d285aaa7150a01628d3689c8c632c0df923634dc16a36acc50bb7f3dedd16b6f58d3fa00080443e7d111bd904092fbeb49d6bcaf23615c746796f8497315d6c673b91e01f8e427800e2342afa8ade6c7442617150e761894dd83cf21d28bcb'

In [73]:
type(byte_data)

bytes

In [74]:
type(ciphertext)

bytes

In [75]:
ciphertext

b"\x04^'\x0fX\xe5\r>H\x1d\xa1\x01)\xc3o\xf0Y\xc4\x94h\xec\xcd\xe5*5\x1c\xf2~\xc3\x0b\x1c9B\xf2ZB%m\xae\xeb<j<\x90\x82\xd1\xfbp\xc9Q\x04Q$oN`^<\xf1\xfb|\xec\xee\xb2\xe0\x8bN\xb9E\xef\x9ad:\x85\xeaF\xa8\xf0\xa3\xf4\x18\xe7T\xb7 /\x97\xbe=J\xb4yT\x8fY\xc8\xdb\x9e\xe8\ne\r oKD\xae\xf25\\(\x1b\xef\x9b\xb7\xa7}\x7f{\xe3\x86\x06\x10x\x8a\xf3i\xc6\xc9p\xa4\xf8\xc0\xb2.\xea\xc5\x19\xa3\x01r\x86\xe0}\x15I\xdf=\xc6^\x16=?\xfc{\xbd<\xfa\x82\x9e\x95\xc0\xe3-9\xe8u\x7f\xc9q$??}\xcd\x97\xfb1y\xd2\x85\xaa\xa7\x15\n\x01b\x8d6\x89\xc8\xc62\xc0\xdf\x9264\xdc\x16\xa3j\xccP\xbb\x7f=\xed\xd1koX\xd3\xfa\x00\x08\x04C\xe7\xd1\x11\xbd\x90@\x92\xfb\xebI\xd6\xbc\xaf#a\\tg\x96\xf8Is\x15\xd6\xc6s\xb9\x1e\x01\xf8\xe4'\x80\x0e#B\xaf\xa8\xad\xe6\xc7D&\x17\x15\x0ev\x18\x94\xdd\x83\xcf!\xd2\x8b\xcb"

In [76]:
# Convert the hex string back into a byte array
byte_array = bytes.fromhex(decompressed_text)

# Print the result as a byte string
print(byte_array)

b"\x04^'\x0fX\xe5\r>H\x1d\xa1\x01)\xc3o\xf0Y\xc4\x94h\xec\xcd\xe5*5\x1c\xf2~\xc3\x0b\x1c9B\xf2ZB%m\xae\xeb<j<\x90\x82\xd1\xfbp\xc9Q\x04Q$oN`^<\xf1\xfb|\xec\xee\xb2\xe0\x8bN\xb9E\xef\x9ad:\x85\xeaF\xa8\xf0\xa3\xf4\x18\xe7T\xb7 /\x97\xbe=J\xb4yT\x8fY\xc8\xdb\x9e\xe8\ne\r oKD\xae\xf25\\(\x1b\xef\x9b\xb7\xa7}\x7f{\xe3\x86\x06\x10x\x8a\xf3i\xc6\xc9p\xa4\xf8\xc0\xb2.\xea\xc5\x19\xa3\x01r\x86\xe0}\x15I\xdf=\xc6^\x16=?\xfc{\xbd<\xfa\x82\x9e\x95\xc0\xe3-9\xe8u\x7f\xc9q$??}\xcd\x97\xfb1y\xd2\x85\xaa\xa7\x15\n\x01b\x8d6\x89\xc8\xc62\xc0\xdf\x9264\xdc\x16\xa3j\xccP\xbb\x7f=\xed\xd1koX\xd3\xfa\x00\x08\x04C\xe7\xd1\x11\xbd\x90@\x92\xfb\xebI\xd6\xbc\xaf#a\\tg\x96\xf8Is\x15\xd6\xc6s\xb9\x1e\x01\xf8\xe4'\x80\x0e#B\xaf\xa8\xad\xe6\xc7D&\x17\x15\x0ev\x18\x94\xdd\x83\xcf!\xd2\x8b\xcb"


In [77]:
decrypted_message = decrypt(private_key_hex, byte_array)
decrypted_message.decode()

'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup'

In [78]:
im = cv2.imread("/content/stego-image/lsb_w_ECC_LZW_package.png")
steg = LSBSteg(im)
bytess = steg.decode_text()
print(bytess.encode())

b'\x18\r\x06\xc2\xa6Q\xc2\x90\xc3\x9c`f\x1a\xc2\x8e\x0c\xc2\xa3Q\xc2\x81\xc2\x90fe\x1a\x0e\x06&C\x08\xc3\x84`1\x19\x0eLc1\xc2\xb1\xc2\x98\xc3\x8c0\x1a\xc3\x86\xc3\x86\xc2\x83\xc2\x91\xc2\xa0\xc3\x9a\x16c1\xc2\x99!\xc2\x83#\x08\xc3\x8cj11\xc2\x99\xc2\xa0\xc3\x86X\xc3\xa0\xc3\x80\xc3\x851\x19\xc3\x89FS1\xc2\xa9\xc2\x84h2\x19\rF\xc3\x91S)\xc2\x94\xc3\x8431\xc2\x8d\xc2\xa5\xc3\x861\xc3\x88\xc3\x80p22\x0cL\xc3\x86(=6a\x02\xc2\x98\x0c\xc2\xa4\xc3\x86a\xc2\xa1\xc2\x94m!2\xc3\x92L\xc3\x95:\xc2\xa9\xc2\x8ekF1\x0c\xc2\x8c\xc2\xb4\xc3\xb3\x15x\xc3\x85%\xc2\x82\x19\xc2\x87&\x11\xc2\xb0\xc3\x90fa\x1cA\'\xc3\xb4\xc2\xb1\xc3\x84\xc2\x82]]\x18\xc3\x82\xc3\x86\xc3\xa3Q\xc2\xa5Td0\xc2\x9e\x0eF\xc3\xa6+\x11\xc2\x90ha\xc2\xb7\r\xc3\x878[\xc3\xb4\xc2\x88\xc3\x8682\\(\xc3\x83\xc2\x81\xc2\x85\xc3\x96\x1ad\xc3\x84G\xc2\xb0\xc3\x83L\xc2\x81\xc2\x96g/\x1a\xc2\x98\xc3\x86Q<m\xc3\x8c\xc3\x85U0\xc2\x8d\xc3\x86\xc3\xa6A\xc2\xb9\xc2\x9b\x19b\x1cX\x06\xc3\x91q\xc2\xb8\xc3\xa0pa3Gccjl\x1e~f\x1c\x18\xc3\xa6\xc3\xb4\x1a

# LSB With Huffman Encoding

In [79]:
import heapq
from collections import Counter

# Class for Huffman Tree Nodes
class HuffmanNode:
    def __init__(self, char=None, freq=0, left=None, right=None):
        self.char = char
        self.freq = freq
        self.left = left
        self.right = right

    def __lt__(self, other):
        return self.freq < other.freq

# Function to build the Huffman Tree
def build_huffman_tree(frequency):
    heap = [HuffmanNode(char, freq) for char, freq in frequency.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        node1 = heapq.heappop(heap)
        node2 = heapq.heappop(heap)
        merged = HuffmanNode(freq=node1.freq + node2.freq, left=node1, right=node2)
        heapq.heappush(heap, merged)

    return heap[0]  # Return the root of the tree

# Function to generate Huffman codes from the Huffman Tree
def generate_huffman_codes(node, current_code="", huffman_codes={}):
    if node is None:
        return

    if node.char is not None:
        huffman_codes[node.char] = current_code

    generate_huffman_codes(node.left, current_code + "0", huffman_codes)
    generate_huffman_codes(node.right, current_code + "1", huffman_codes)

    return huffman_codes

# Function to compress the data using Huffman Encoding
def huffman_compress(data):
    # Count the frequency of each byte
    frequency = Counter(data)

    # Build Huffman Tree
    huffman_tree = build_huffman_tree(frequency)

    # Generate Huffman Codes
    huffman_codes = generate_huffman_codes(huffman_tree)

    # Encode data into its binary representation
    encoded_data = "".join(huffman_codes[byte] for byte in data)

    # Pad the encoded data to make its length a multiple of 8
    padding = 8 - len(encoded_data) % 8
    encoded_data += "0" * padding
    padding_info = "{0:08b}".format(padding)

    # Convert binary data to bytes
    compressed_data = bytearray()
    compressed_data.append(int(padding_info, 2))  # First byte stores the padding information
    for i in range(0, len(encoded_data), 8):
        byte = encoded_data[i:i+8]
        compressed_data.append(int(byte, 2))

    return compressed_data, huffman_tree

# Function to decompress the data using Huffman Encoding
def huffman_decompress(compressed_data, huffman_tree):
    # Convert the compressed data to binary string
    encoded_data = ""
    for byte in compressed_data[1:]:
        encoded_data += "{0:08b}".format(byte)

    # Get the padding information from the first byte
    padding = compressed_data[0]
    encoded_data = encoded_data[:-padding]  # Remove padding

    # Traverse the Huffman tree to decode the binary string
    decoded_data = bytearray()
    node = huffman_tree
    for bit in encoded_data:
        if bit == "0":
            node = node.left
        else:
            node = node.right

        if node.left is None and node.right is None:  # Leaf node
            decoded_data.append(node.char)
            node = huffman_tree

    return decoded_data


# Example Usage of Huffman Compression/Decompression

# Step 1: The original plaintext data (as a byte string)
plaintext = CONTENT.encode()

# Print the original data
print("Original Data (as bytes):", plaintext)
print("Original Data (as string):", plaintext.decode())

# Step 2: Compress the plaintext data using Huffman encoding
compressed_data, huffman_tree = huffman_compress(plaintext)
print("Compressed Data (in bytes):", compressed_data)

# Step 3: Decompress the compressed data back to the original plaintext
decompressed_data = huffman_decompress(compressed_data, huffman_tree)
print("Decompressed Data (as bytes):", decompressed_data)
print("Decompressed Data (as string):", decompressed_data.decode())

# Step 4: Verify that the decompressed data matches the original data
if decompressed_data == plaintext:
    print("Decompression was successful!")
else:
    print("Decompression failed.")


Original Data (as bytes): b'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup'
Original Data (as string): 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup
Compressed Data (in bytes): bytearray(b'\x02\xdfy>\xa8\x7f\x12G\xf7S\x11\x1e\xfb\xedYu\xcf\xc6)\x91\xd52\xe6\xc3\xa3usi\xf9\xd3OD;\xfb\xae\xa6\xf8\xde\xf8\x93\x80\xe1\xef\x03\x87\xed9\x90\xa2;\x10#\xf3\xe0\xdb\xca#\xbctG\xbe\x8f\xe9\xea\xa3%iQ\x9a[\xd1Wc&\xa6}\x835x\x0f\x8e\xd1\x01\xb5\x1c\x19^\xdd\xb2\x01d\xe5\x85J\x8a\xddr\x9e\x96\xcf\xcbF\xd5\xb9\x15>\xad\xd8')
Decompressed Data (as bytes): bytearray(b'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup')

In [80]:
compressed_data.hex()

'02df793ea87f1247f753111efbed5975cfc62991d532e6c3a3757369f9d34f443bfbaea6f8def89380e1ef0387ed3990a23b1023f3e0dbca23bc7447be8fe9eaa32569519a5bd1576326a67d8335780f8ed101b51c195eddb20164e5854a8add729e96cfcb46d5b9153eadd8'

In [81]:
embed_lsb("lsb_w_Huffman.png", compressed_data.hex())

In [82]:
extracted_data = extract_lsb("lsb_w_Huffman.png")
extracted_data

'02df793ea87f1247f753111efbed5975cfc62991d532e6c3a3757369f9d34f443bfbaea6f8def89380e1ef0387ed3990a23b1023f3e0dbca23bc7447be8fe9eaa32569519a5bd1576326a67d8335780f8ed101b51c195eddb20164e5854a8add729e96cfcb46d5b9153eadd8'

In [83]:
calculate_psnr_2("lsb_w_Huffman.png")

PSNR: 77.81371174500211


77.81371174500211

In [84]:
byte_array = bytearray.fromhex(extracted_data)

# Print the bytearray
print(byte_array)

bytearray(b'\x02\xdfy>\xa8\x7f\x12G\xf7S\x11\x1e\xfb\xedYu\xcf\xc6)\x91\xd52\xe6\xc3\xa3usi\xf9\xd3OD;\xfb\xae\xa6\xf8\xde\xf8\x93\x80\xe1\xef\x03\x87\xed9\x90\xa2;\x10#\xf3\xe0\xdb\xca#\xbctG\xbe\x8f\xe9\xea\xa3%iQ\x9a[\xd1Wc&\xa6}\x835x\x0f\x8e\xd1\x01\xb5\x1c\x19^\xdd\xb2\x01d\xe5\x85J\x8a\xddr\x9e\x96\xcf\xcbF\xd5\xb9\x15>\xad\xd8')


In [85]:
decompressed_data = huffman_decompress(byte_array, huffman_tree)
print("Decompressed Data (as bytes):", decompressed_data)

Decompressed Data (as bytes): bytearray(b'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup')


In [86]:
print("Decompressed Data (as string):", decompressed_data.decode())

Decompressed Data (as string): 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup
